# Algorithme de recherche de plus court chemin
*NB 2020-11-28*

Ce notebook vous permet de prendre en main divers algorithmes de recherche. Ces algorithmes appartiennent à deux classes différentes

*   recherche non-informée: 
  * recherche en largeur d'abord (breadth-first search)
  * recherche en profondeux d'abord (depth-first search)
  * uniform-cost ("Dijsktra avec un but")
*   recherche informée (on dispose d'une heuristique):
  * glouton (best-first search)
  * A\*

**Lexique:**
* Frontière: l'ensemble des noeuds actuellement connus et à explorés
* Archive (ou réserve): l'ensemble des noeuds déjà explorés
* Facteur de branchement du problème: le nombre de successeurs possibles à chaque état
* Cout: le passage d'un état à un autre à un coût

**Critères de comparaison** (Russel & Norvig, 2003)
* optimalité: une solution est optimale si elle permet de passer de l'état initial à l'état but avec un coût minimal.
* complétude: l'algorithme trouve-t-il une solution (sous réserve qu'elle existe)?
* qualité: la solution trouvée est-elle optimale?
* complexité de calcul: le temps de calcul nécessaire pour obtenir la solution.
* complexité en espace: l'espace de stockage nécessaire pendant le calcul.




# Définitions de quelques labyrinthes

In [ ]:
from datetime import datetime
from datetime import date

# Labyrinthe
# Légende: case vide (0), mur (1), départ (5), but (6)

mazeCells_8x8_serpentin = [
        [ 0, 0, 0, 1, 0, 0, 0, 1], 
        [ 0, 1, 0, 0, 0, 1, 0, 0], 
        [ 0, 1, 1, 1, 1, 0, 1, 0], 
        [ 5, 0, 0, 0, 0, 0, 1, 1], 
        [ 0, 1, 1, 1, 1, 0, 1, 6], 
        [ 0, 0, 1, 0, 0, 0, 1, 0], 
        [ 0, 1, 1, 0, 1, 1, 1, 0], 
        [ 0, 0, 1, 0, 0, 0, 0, 0], 
        ]

mazeCells_8x8_bloc = [
        [ 0, 0, 0, 0, 0, 0, 0, 0], 
        [ 0, 0, 0, 1, 1, 0, 0, 0], 
        [ 0, 0, 0, 0, 1, 0, 0, 0], 
        [ 5, 0, 0, 0, 1, 0, 0, 0], 
        [ 0, 0, 0, 0, 1, 0, 0, 6], 
        [ 0, 0, 0, 0, 1, 0, 0, 0], 
        [ 0, 0, 0, 1, 1, 0, 0, 0], 
        [ 0, 0, 0, 0, 0, 0, 0, 0], 
        ]

mazeCells_5x5_0 = [ 
        [ 0, 5, 0, 0, 0], 
        [ 0, 1, 0, 1, 0],
        [ 0, 1, 0, 0, 1],
        [ 0, 1, 1, 0, 0],
        [ 0, 0, 0, 0, 6]  
        ]

mazeCells_5x5_1 = [ 
        [ 0, 5, 0, 1, 0], 
        [ 0, 1, 0, 1, 0],
        [ 0, 1, 0, 0, 0],
        [ 0, 1, 1, 1, 1],
        [ 0, 0, 0, 0, 6]  
        ]

# Table des heuristiques

mazeHeuristicsCost_naive = [  # distance de manhattan au but
        [11,10, 9, 8, 7, 6, 5, 4], 
        [10, 9, 8, 7, 6, 5, 4, 3], 
        [ 9, 8, 7, 6, 5, 4, 3, 2], 
        [ 8, 7, 6, 5, 4, 3, 2, 1], 
        [ 7, 6, 5, 4, 3, 2, 1, 0],
        [ 8, 7, 6, 5, 4, 3, 2, 1], 
        [ 9, 8, 7, 6, 5, 4, 3, 2], 
        [10, 9, 8, 7, 6, 5, 4, 3]  
        ]

mazeHeuristicsCost_truebest = [  # distance réelle au but en prenant en compte les murs
        [21,22,23,-1,27,28,29,-1], 
        [20,-1,24,25,26,-1,30,31], 
        [19,-1,-1,-1,-1,14,-1,32], 
        [18,17,16,15,14,13,-1,-1], 
        [19,-1,-1,-1,-1,12,-1, 0], 
        [20,21,-1, 9,10,11,-1, 1], 
        [21,-1,-1, 8,-1,-1,-1, 2], 
        [22,23,-1, 7, 6, 5, 4, 3], 
        ]

mazeHeuristicsCost_vaguebest = [  # distance vaguement réelle
        [ 4, 4, 4,-1, 4, 4, 4,-1], 
        [ 4,-1, 4, 4, 4,-1, 4, 4], 
        [ 4,-1,-1,-1,-1, 4,-1, 4], 
        [ 4, 4, 4, 4, 4, 4,-1,-1], 
        [ 4,-1,-1,-1,-1, 4,-1, 0], 
        [ 3, 2,-1, 4, 4, 4,-1, 1], 
        [ 3,-1,-1, 4,-1,-1,-1, 2], 
        [ 3, 3,-1, 5, 6, 5, 4, 3], 
        ]

mazeHeuristicsCost_5x5 = [ # distance de manhattan au but
        [ 8, 7, 6, 5, 4], 
        [ 7, 6, 5, 4, 3],
        [ 6, 5, 4, 3, 2],
        [ 5, 4, 3, 2, 1],
        [ 4, 3, 2, 1, 0]  
        ]

print("\n",date.today(), datetime.now().strftime("%H:%M:%S"),"GMT") # timestamp is greenwich time
print("OK.")



 2021-10-13 15:25:45 GMT
OK.


# Initialisation des données

In [ ]:

# affiche (lisiblement) le labyrinthe
def displayMaze(m):
  d = {0:"·", 1:"#", 5:"s", 6:"g"}
  for l in m:
    s = ""
    for c in l:
      if c in d:
        s+=d[c]
      else:
        s+="?"
    print (s)
  return

# localise la position de départ
def locateStart (m):
  for i in range (len(m)):
    for j in range (len(m[i])):
      if m[i][j] == 5:
        return ((i,j))
        break
  print ("[erreur] Pas de position de départ")
  return None

# localise le (ou les) but(s)
def locateGoals (m):
  l = []
  for i in range (len(m)):
    for j in range (len(m[i])):
      if m[i][j] == 6:
        l.append((i,j))
  if not l:
    print ("[erreur] Pas cellule objectif trouvée.")
  return l

# construit une table de hachage contenant les états d'un labyrinthe
def mapMazeMatrixToGraph (m):
  g={}
  for i in range (len(m)):
    for j in range (len(m[i])):
      if m[i][j] != 1:
        values = []
        if i>0:
          if m[i-1][j] != 1:
            values.append((i-1,j))
        if i<len(m)-1:
          if m[i+1][j] != 1:
            values.append((i+1,j))
        if j>0:
          if m[i][j-1] != 1:
            values.append((i,j-1))
        if j<len(m[0])-1:
          if m[i][j+1] != 1:
            values.append((i,j+1))
        if values != []:
          g[(i,j)] = values
  return g

def mapHeuristicMatrixToGraph (m):
  g={}
  for i in range (len(m)):
    for j in range (len(m[i])):
      if m[i][j] != -1:
        g[(i,j)] = m[i][j]
  return g

print("\n",date.today(), datetime.now().strftime("%H:%M:%S"),"GMT") # timestamp is greenwich time
print("OK.")


 2021-10-13 15:25:54 GMT
OK.


# Chargement du labyrinthe

Astuce: modifiez les deux premières lignes pour changer de labyrinthe

In [ ]:
mazeCells = mazeCells_8x8_serpentin
mazeHeuristicsCost = mazeHeuristicsCost_vaguebest

print ("Labyrinthe:")
#print (mazeCells)
displayMaze(mazeCells)

start = locateStart(mazeCells)
print ("\nPosition de départ:",start)

goals = locateGoals(mazeCells)
print ("\nPosition du/des but/s:",goals)

mazeGraph = mapMazeMatrixToGraph(mazeCells)
#print ("\nGraphe d'états correspondant:")
#for d in mazeGraph:
#  print (d," : ",mazeGraph[d])

heuristicDict = mapHeuristicMatrixToGraph(mazeHeuristicsCost)
#print ("\nValeurs de l'heuristique par état:")
#for d in heuristicDict:
#  print (d," : ",heuristicDict[d])


print("\n",date.today(), datetime.now().strftime("%H:%M:%S"),"GMT") # timestamp is greenwich time
print("OK.")

Labyrinthe:
···#···#
·#···#··
·####·#·
s·····##
·####·#g
··#···#·
·##·###·
··#·····

Position de départ: (3, 0)

Position du/des but/s: [(4, 7)]

 2021-10-13 15:26:13 GMT
OK.


# Recherche non-informée (1): parcours en **largeur** d'abord

*Breadth-first search algorithm (BFS)*

In [ ]:
# Recherche de chemin en largeur d'abord (breadth-first search)

frontier = []
visited = [] 

print ("start:", start)
print ("goal :", goals[0])
print ("graph:",mazeGraph)

frontier.append(start) 

it = 0

while len(frontier) > 0:
  it = it + 1
  print ("Frontière :",frontier)
  print ("Archive   :",visited)
  current_state = frontier.pop(0)
  print ("Selection :",current_state)
  for successor in mazeGraph[current_state]:
    print ("Successeur:",successor)
    if successor not in frontier and successor not in visited:
      frontier.append(successor)
  visited.append(current_state)
  if goals[0] in frontier:
    print ("Trouve!")
    break

#print ("Frontière:",frontier)
#print ("Archive  :",visited)
print ("Itérations:",it)

start: (3, 0)
goal : (4, 7)
graph: {(0, 0): [(1, 0), (0, 1)], (0, 1): [(0, 0), (0, 2)], (0, 2): [(1, 2), (0, 1)], (0, 4): [(1, 4), (0, 5)], (0, 5): [(0, 4), (0, 6)], (0, 6): [(1, 6), (0, 5)], (1, 0): [(0, 0), (2, 0)], (1, 2): [(0, 2), (1, 3)], (1, 3): [(1, 2), (1, 4)], (1, 4): [(0, 4), (1, 3)], (1, 6): [(0, 6), (1, 7)], (1, 7): [(2, 7), (1, 6)], (2, 0): [(1, 0), (3, 0)], (2, 5): [(3, 5)], (2, 7): [(1, 7)], (3, 0): [(2, 0), (4, 0), (3, 1)], (3, 1): [(3, 0), (3, 2)], (3, 2): [(3, 1), (3, 3)], (3, 3): [(3, 2), (3, 4)], (3, 4): [(3, 3), (3, 5)], (3, 5): [(2, 5), (4, 5), (3, 4)], (4, 0): [(3, 0), (5, 0)], (4, 5): [(3, 5), (5, 5)], (4, 7): [(5, 7)], (5, 0): [(4, 0), (6, 0), (5, 1)], (5, 1): [(5, 0)], (5, 3): [(6, 3), (5, 4)], (5, 4): [(5, 3), (5, 5)], (5, 5): [(4, 5), (5, 4)], (5, 7): [(4, 7), (6, 7)], (6, 0): [(5, 0), (7, 0)], (6, 3): [(5, 3), (7, 3)], (6, 7): [(5, 7), (7, 7)], (7, 0): [(6, 0), (7, 1)], (7, 1): [(7, 0)], (7, 3): [(6, 3), (7, 4)], (7, 4): [(7, 3), (7, 5)], (7, 5): [(7, 4), (

# Recherche non-informée (2): parcours en **profondeur** d'abord

*Depth-first search algorithm*

In [ ]:
# Recherche de chemin en profondeur d'abord (depth-first search)

frontier = [] # or: queue
visited = [] # 

print ("start:", start)
print ("goal :", goals[0])
print ("graph:",mazeGraph)

frontier.append(start) # [0] state [1] trace from start
it = 0
terminate = False

while len(frontier) > 0 and terminate == False:
  it = it + 1
  print ("Frontière :",frontier)
  print ("Archive   :",visited)
  
  current_state = frontier[0]
  next_candidate = None
  print ("Selection :",current_state)

  for successor in mazeGraph[current_state]: # y a t'il un successeur à explorer?
    if successor not in frontier and successor not in visited:
      print ("Successeur:",successor)
      next_candidate = successor
      break
  if next_candidate == None: # on enlève le noeud complètement visité
    visited.append(current_state)
    frontier.pop(0)
  else: # on ajoute le nouveau pour l'explorer à son tour (en profondeur)
    frontier.insert(0,next_candidate)
    if next_candidate in goals:
      print ("Trouvé!")
      terminate = True
      break

print ("Itérations:",it)

---

# Recherche informée: généralités

On fait maintenant l'hypothèse qu'on dispose d'une estimation de la qualité d'un état particulier. On utilise une heuristique pour calculer cette estimation à partir d'un état donné. Par exemple, dans le cas d'un robot cherchant la sortie d'un labyrinthe, la distance euclidienne à la sortie est une heuristique candidate. Cette estimation est partiellement vraie, car la présence d'obstacles peut rendre la distance *réelle* à parcourir plus grande.


# Recherche informée: algorithme glouton

*Greedy best-first search algorithm*

Il s'agit tout simplement d'étendre le noeud le plus "prometteur" selon l'heuristique.


In [ ]:
# Recherche informée: algorithme glouton

frontier = [] 
visited = [] 

print ("start:", start)
print ("goal :", goals[0])
print ("graph:", mazeGraph)
print ("h-graph:", heuristicDict)

frontier.append(start) 
it = 0
terminate = False

while len(frontier) > 0 and terminate == False:
  it = it + 1
  print ("Frontière :",frontier)
  print ("Archive   :",visited)
  
  # choisi l'état à étendre (i.e. celui qui a la valeur heuristique la plus faible)
  selection_index = 0
  best_heuristic = 999999
  for i in range(len(frontier)):
    if heuristicDict[frontier[i]] < best_heuristic:
       selection_index = i
       best_heuristic = heuristicDict[frontier[i]]
  print ("Selection: ", frontier[selection_index])

  # enregistre les successeurs
  for successor in mazeGraph[frontier[selection_index]]:
    if successor not in frontier and successor not in visited:
      frontier.append(successor)
      if successor in goals:
        print ("Trouvé!")
        terminate = True
        break

  # met l'état courant dans l'archive des états visités
  visited.append(frontier[selection_index])
  frontier.pop(selection_index)


print ("Itérations:",it)

start: (3, 0)
goal : (4, 7)
graph: {(0, 0): [(1, 0), (0, 1)], (0, 1): [(0, 0), (0, 2)], (0, 2): [(1, 2), (0, 1)], (0, 4): [(1, 4), (0, 5)], (0, 5): [(0, 4), (0, 6)], (0, 6): [(1, 6), (0, 5)], (1, 0): [(0, 0), (2, 0)], (1, 2): [(0, 2), (1, 3)], (1, 3): [(1, 2), (1, 4)], (1, 4): [(0, 4), (1, 3)], (1, 6): [(0, 6), (1, 7)], (1, 7): [(2, 7), (1, 6)], (2, 0): [(1, 0), (3, 0)], (2, 5): [(3, 5)], (2, 7): [(1, 7)], (3, 0): [(2, 0), (4, 0), (3, 1)], (3, 1): [(3, 0), (3, 2)], (3, 2): [(3, 1), (3, 3)], (3, 3): [(3, 2), (3, 4)], (3, 4): [(3, 3), (3, 5)], (3, 5): [(2, 5), (4, 5), (3, 4)], (4, 0): [(3, 0), (5, 0)], (4, 5): [(3, 5), (5, 5)], (4, 7): [(5, 7)], (5, 0): [(4, 0), (6, 0), (5, 1)], (5, 1): [(5, 0)], (5, 3): [(6, 3), (5, 4)], (5, 4): [(5, 3), (5, 5)], (5, 5): [(4, 5), (5, 4)], (5, 7): [(4, 7), (6, 7)], (6, 0): [(5, 0), (7, 0)], (6, 3): [(5, 3), (7, 3)], (6, 7): [(5, 7), (7, 7)], (7, 0): [(6, 0), (7, 1)], (7, 1): [(7, 0)], (7, 3): [(6, 3), (7, 4)], (7, 4): [(7, 3), (7, 5)], (7, 5): [(7, 4), (

# Recherche informée: algorithme A*

L'algorithme A\* est une variation de l'algorithme précédent (best-first). En plus du coût estimée par l'heuristique, A\*  intègre aussi le le cout du chemin jusqu'au noeud considéré. Le calcul du "score" d'un noeud est le suivant:

  **f(n) = g(n) + h(n)**

avec:
*   f(n): score du noeud *n* (que l'on espère faible)
*   g(n): coût du chemin jusqu'au noeud *n*
*   h(n): coût estimé du chemin depuis le noeud *n* jusqu'au but

L'heuristique doit avoir les deux propriétés suivantes pour que A\* retourne une solution optimale:

1.   **Admissibilité**: une heuristique est admissible lorsqu'elle ne surestime jamais la distance à l'état but (on parle aussi d'heuristique minorante)

2.   **Consistence**: une heuristique est consistante ssi pour tout état m et tout fils de n de m, on a: h(m)≤cost(m,n)+h(n) -- Autrement dit, les valeurs de f(n) sont non-décroissantes sur les chemins depuis la racine.

Référence: Rina Dechter, Judea Pearl (1985) *Generalized best-first search strategies and the optimality of A\**. Journal of the ACM. 32(3):505–536. 

In [ ]:
# Recherche informée: algorithm A*

frontier = [] # or: queue
visited = [] # 

print ("start:", start)
print ("goal :", goals[0])
print ("graph:", mazeGraph)
print ("h-graph:", heuristicDict)

frontier.append( start )  
dict_state = {}
dict_state[start] = [ 0, 0, []] # score, cout depuis le départ, trace
print ("dict_state:",dict_state) # DEBUG
it = 0
terminate = False

while len(frontier) > 0 and terminate == False:
  it = it + 1
  print ("Frontière :",frontier)
  print ("Archive   :",visited)

  # choisi la cellule à étendre (i.e. celle qui a le score le plus faible)
  selection_index = 0
  selection_score = 999999
  for i in range(len(frontier)):
    if dict_state[frontier[i]][0] < selection_score:
       selection_index = i
       selection_score = dict_state[frontier[i]][0]
  
  # s'agit il d'une cellule cible?
  if frontier[selection_index] in goals:
    print("Trouvé!")
    print("Chemin:",dict_state[frontier[selection_index]][2]+[frontier[selection_index]])
    terminate = True
    break

  # calcul le score de chacun de ses successeurs -- pour chacun, on l'enregistre (ou on le met à jour dans la frontière *que* s'il a un meilleur score)
  for successor in mazeGraph[ frontier[selection_index] ]:
    if successor not in visited: 
      successor_cost  = dict_state[frontier[selection_index]][0] + 1
      successor_score = successor_cost + heuristicDict[successor]
      if successor not in frontier:
        frontier.append(successor)
        dict_state[successor] = [ successor_score, successor_cost, dict_state[frontier[selection_index]][2]+[frontier[selection_index]] ]
      elif dict_state[successor][0] > successor_score:
          dict_state[successor] = [ successor_score, successor_cost, dict_state[frontier[selection_index]][2]+[frontier[selection_index]] ]
  
  # on archive le noeud qu'on vient d'étendre
  visited.append(frontier[selection_index])
  frontier.pop(selection_index)

print ("Itérations:",it)

start: (3, 0)
goal : (4, 7)
graph: {(0, 0): [(1, 0), (0, 1)], (0, 1): [(0, 0), (0, 2)], (0, 2): [(1, 2), (0, 1)], (0, 4): [(1, 4), (0, 5)], (0, 5): [(0, 4), (0, 6)], (0, 6): [(1, 6), (0, 5)], (1, 0): [(0, 0), (2, 0)], (1, 2): [(0, 2), (1, 3)], (1, 3): [(1, 2), (1, 4)], (1, 4): [(0, 4), (1, 3)], (1, 6): [(0, 6), (1, 7)], (1, 7): [(2, 7), (1, 6)], (2, 0): [(1, 0), (3, 0)], (2, 5): [(3, 5)], (2, 7): [(1, 7)], (3, 0): [(2, 0), (4, 0), (3, 1)], (3, 1): [(3, 0), (3, 2)], (3, 2): [(3, 1), (3, 3)], (3, 3): [(3, 2), (3, 4)], (3, 4): [(3, 3), (3, 5)], (3, 5): [(2, 5), (4, 5), (3, 4)], (4, 0): [(3, 0), (5, 0)], (4, 5): [(3, 5), (5, 5)], (4, 7): [(5, 7)], (5, 0): [(4, 0), (6, 0), (5, 1)], (5, 1): [(5, 0)], (5, 3): [(6, 3), (5, 4)], (5, 4): [(5, 3), (5, 5)], (5, 5): [(4, 5), (5, 4)], (5, 7): [(4, 7), (6, 7)], (6, 0): [(5, 0), (7, 0)], (6, 3): [(5, 3), (7, 3)], (6, 7): [(5, 7), (7, 7)], (7, 0): [(6, 0), (7, 1)], (7, 1): [(7, 0)], (7, 3): [(6, 3), (7, 4)], (7, 4): [(7, 3), (7, 5)], (7, 5): [(7, 4), (

# Quelques exercices pour aller plus loin

* Pour chacun des trois premiers algorithmes, construisez un labyrinthe pour lequel il domine les autres. 
* Pour l'algorithme glouton, construisez un labyrinthe pour lequel il donne la solution optimale
* Pour l'algorithme glouton, construisez un labyrinthe pour lequel il donne la pire solution. Comment définissez-vous la pire solution?
* Pour l'algorithme A\*, une heuristique qui renvoie tout le temps 0 est elle acceptable?
* Pour l'algorithme A\*, vérifiez que les heuristiques proposées dans le code respectent bien les propriétés requises pour garantir que A\* renvoie la solution optimale
* Comparez l'algorithme glouton et le A\* sur la labyrinthe "mazeCells_8x8_serpentin". Proposez (et testez) une modification de la définition de ce labyrinthe pour lequel A\* serait bien plus efficace.
* Pour l'algorithme A\*, proposez d'autres heuristiques et illustrez les sur des labyrinthes de votre création. 
* Il est possible d'améliorer l'algorithme de recherche en profondeur d'abord en limitant la profondeur de la recherche. A partir du point de départ, la recherche en profondeur est limitée. Lorsque tous les noeuds à cette profondeur ont été visités, la profondeux maximale est étendue de 1, et ainsi de suite. Implémentez cet algorithme ("depth-first search with iterative deepening")
* L'amélioration précédente est aussi applicable à l'algorithme A\*. Implémentez cette version de l'algorithme A\*